In [2]:
import os
import requests
from typing import Type  # 타입 주석에 사용되는 Type 클래스 import
from langchain.chat_models import ChatOpenAI  # OpenAI 언어 모델 사용
from langchain.tools import BaseTool  # Langchain BaseTool 클래스 import
from pydantic import BaseModel, Field  # Pydantic의 BaseModel, Field 사용
from langchain.agents import initialize_agent, AgentType  # 에이전트 초기화 및 에이전트 유형 사용
from langchain.utilities import DuckDuckGoSearchAPIWrapper,WikipediaAPIWrapper
from langchain.document_loaders import WebBaseLoader



from dotenv import load_dotenv
import os

# .env 파일에서 API 키를 불러옵니다.
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# OpenAI 언어 모델을 설정합니다.
llm = ChatOpenAI(temperature=0.1, api_key=api_key, model_name="gpt-4o-mini")


def save_to_txt(content, filename="scraped_content.txt"):
    file_dir = "./.scrapped/files"
    os.makedirs(file_dir, exist_ok=True) 
    ile_path = f"./.scrapped/files/{filename}"  # 파일 경로 설정
    with open(ile_path, 'w', encoding='utf-8') as file:
        file.write(content)
    return f"Content saved to {filename}"

# 주식 심볼을 검색하기 위한 도구에 필요한 입력 스키마 정의
class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="검색할 쿼리를 입력하세요. 예시 쿼리: 애플 회사의 주식 심볼"
    )

# 주식 심볼을 검색하는 도구 클래스 정의
class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"  # 도구 이름
    description = """
    이 도구는 입력에 대한 검색을 하고 컨텐츠를 가져오기 위한 도구입니다.
    쿼리를 입력받아 사용됩니다.
    """  # 도구 설명
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    # DuckDuckGo API를 통해 검색 실행
    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()  # DuckDuckGo API 사용
        return ddg.run(query)  # 검색 결과 반환

# 주식 심볼을 검색하기 위한 도구에 필요한 입력 스키마 정의
class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="검색할 쿼리를 입력하세요. 예시 쿼리: the XZ backdoor"
    )

# 위키피디아
class WikipediaSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="검색할 쿼리를 입력하세요. 예시 쿼리: thee XZ backdoor"
    )


class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"  # 도구 이름
    description = """
    이 도구는 입력에 대한 검색을 하고 컨텐츠를 가져오기 위한 도구입니다.
    쿼리를 입력받아 사용됩니다.
    """  # 도구 설명
    args_schema: Type[WikipediaSearchToolArgsSchema] = WikipediaSearchToolArgsSchema

    def _run(self, query):
        ddg = WikipediaAPIWrapper()  #
        return ddg.run(query)  # 검색 결과 반환

class WebScrapingToolArgsSchema(BaseModel):
    url: str = Field(
        description="스크랩할 웹사이트의 URL을 입력하세요."
    )

# 스크랩퍼
class WebScrapingTool(BaseTool):
    name = "WebScrapingTool"
    description = """
    이 도구는 주어진 웹사이트 URL에서 텍스트를 스크랩합니다.
    """  # 도구 설명
    args_schema: Type[WebScrapingToolArgsSchema] = WebScrapingToolArgsSchema

    def _run(self, url):
        loader = WebBaseLoader(url)
        docs = loader.load()
        return docs[0].page_content  # 첫 번째 문서의 텍스트 반환


# 에이전트를 초기화합니다.
agent = initialize_agent(
    llm=llm,  # 언어 모델 설정
    verbose=True,  # 상세 로그 출력
    agent=AgentType.OPENAI_FUNCTIONS,  # 에이전트 유형 설정 (OpenAI 함수형 에이전트)
    handle_parsing_errors=True,  # 파싱 오류 처리 활성화
    tools=[
        StockMarketSymbolSearchTool(),  # 주식 심볼 검색 도구 추가
        WikipediaSearchTool()
    ],
)

# 사용자의 질문을 한국어로 작성
prompt = "XZ backdoor에 대해서 검색해주세요."

# 에이전트를 사용하여 질문 실행
result = agent.invoke(prompt)

scraped_content = result["output"]
save_result = save_to_txt(scraped_content)
print(save_result)




> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.

Page: XZ Utils
Summary: XZ Utils (previously LZM